# 作业分别用自己编写函数实现和用Scikit-learn实现

## 自己编写函数实现

In [1]:
import numpy as np
import pandas as pd
import operator
from numpy import *
import os

#读取坐标点数据
def read_data(filename):
    data=[]
    with open('./%s'%filename) as f:
        for line in f:
            line_data=[x for x in line.split(',')]
            data.append(line_data)
    data=np.array(data,dtype='float32')
    return(data)

#读取坐标点对应的方向数据
def read_label(filename):
   #待实现
    label=[]
    with open('./%s'%filename) as f:
        lines=f.readlines()
#         print(lines)
        for line in lines:
            label.append(line.strip('\n'))        
    return(label)  

#读取数据
trainingData = read_data('train_states.txt')
trainingLabel = read_label('train_labels.txt')
testData = read_data('test_states.txt')
testLabel = read_label('test_labels.txt')

#计算目标点和所有训练点之间的欧氏距离
def euclid_distance(x1, x2, dim):
    distance = 0
    sum = 0
    for x in range(dim): #遍历所有的训练点维度
        distance = pow((x1[x] - x2[x]), 2)#向量值平方
        sum = sum + distance
    #print(sum)
    return math.sqrt(sum)/2


# 计算所有目标点和所有训练点之间的距离，并选出K个最近邻
def get_Neighbors(training_set, test_set, k):
    DistTable=[]#点及点与测试点的距离
    index=[]#与测试点的距离
    for i in range(len(training_set)):
        dist=euclid_distance(test_set,training_set[i],len(training_set[0]))
        DistTable.append([training_set[i],dist])
        index.append(dist)
    # 对所有的距离进行排序
    DistTable.sort(key=operator.itemgetter(1))
#     print(DistTable.sort(key=operator.itemgetter(1)))#*****
    index = array(index).argsort() #将list转换为array，以便调用argsort,数值从小到大的顺序取出
    neighbors = []
    # 返回k个最近邻
    for x in range(k):
        neighbors.append(DistTable[x][0])
    index = index[:k]
    return index

# 对k个近邻进行合并，返回value最大的key
def get_label(index):
    label_count = {}
    for x in index:
        label = trainingLabel[x]
#         print('label_count:',label_count,'label:',label)
        if label not in label_count:
            label_count[label] = 1
        else:
            label_count[label] += 1
            
            # 排序
    sorted_label = sorted(label_count.items(), key=operator.itemgetter(1), reverse=True)
    print(sorted_label)
    print(sorted_label[0][0])
    return sorted_label[0][0]

#测试并统计正确率
def predict():
#     trainingSet =trainingData
#     testSet =testData
    trainingSet = [[x[3]] for x in trainingData]
    testSet = [[x[3]] for x in testData]
    m = len(testSet)
#     print(m)
    count = 0
    for i in range(m):
        index = get_Neighbors(trainingSet, testSet[i], 50)
        label = get_label(index)
        if(label == testLabel[i]):
            count+=1
            print("\033[1;32m%s\033[0m" % 'correct')#字符串前缀：\033[显示方式;字体色;背景色m
#             print('correct')
        else:
            print(trainingSet[i])
            print(testLabel[i])
            print("\033[1;31m%s\033[0m" % 'error')
    print("correct rate：", count/m)


#调用predict,显示预测结果
predict()



[('right', 50)]
right
correct
[('keep', 41), ('left', 9)]
keep
correct
[('left', 50)]
left
correct
[('left', 50)]
left
correct
[('keep', 41), ('left', 9)]
keep
correct
[('left', 49), ('keep', 1)]
left
correct
[('left', 50)]
left
correct
[('keep', 30), ('right', 14), ('left', 6)]
keep
correct
[('left', 50)]
left
correct
[('right', 30), ('keep', 20)]
right
correct
[('keep', 34), ('right', 8), ('left', 8)]
keep
correct
[('left', 50)]
left
correct
[('left', 50)]
left
correct
[('right', 50)]
right
correct
[('keep', 33), ('left', 9), ('right', 8)]
keep
[0.018020319]
left
error
[('keep', 33), ('left', 17)]
keep
[-1.7738346]
left
error
[('right', 50)]
right
correct
[('keep', 39), ('right', 11)]
keep
[-1.3596292]
right
error
[('right', 50)]
right
correct
[('keep', 29), ('left', 21)]
keep
[-1.0674729]
left
error
[('left', 50)]
left
correct
[('right', 50)]
right
correct
[('right', 50)]
right
correct
[('left', 48), ('keep', 2)]
left
correct
[('keep', 40), ('left', 10)]
keep
correct
[('keep', 34), 

* 使用trainingData所有数据，准确率最高为0.54
* 仅使用trainingData的加速度数据，准确率最高的为：n_neighbors=7,weights='uniform',准确率为：0.848

##  Scikit-learn实现

In [2]:
# from sklearn.neighbors import NearestNeighbors
from sklearn import neighbors
import pandas as pd
import numpy as np
%matplotlib inline


#Read data
trainingData = pd.read_table('./train_states.txt',sep=',',header=None)
trainingLabel = pd.read_table('./train_labels.txt',header=None)
print('Shape of trainingData:',trainingData.shape,'\n','Shape of trainingLabel',trainingLabel.shape)
testData = pd.read_table('test_states.txt',sep=',',header=None)
testLabel = pd.read_table('test_labels.txt',header=None)
print('Shape of testData:',testData.shape,'\n','Shape of testLabel',testLabel.shape)

#Plot
# trainingData.plot(kind='scatter',x=0,y=1)
# testData.plot(kind='scatter',x=0,y=1)

#KNN
for n_neighbors in range(1,10):
    for  weights in ['uniform','distance']:
        model = neighbors.KNeighborsClassifier(n_neighbors, weights=weights)
        model.fit(trainingData, ravel(trainingLabel))
        test_accuracy = model.score(testData, testLabel)
        print('All data:',n_neighbors,weights,test_accuracy)

for n_neighbors in range(1,10):
    for  weights in ['uniform','distance']:
        model = neighbors.KNeighborsClassifier(n_neighbors, weights=weights)
        model.fit(trainingData.iloc[:,[2,3]], ravel(trainingLabel))
        test_accuracy = model.score(testData.iloc[:,[2,3]], testLabel)
        print(n_neighbors,weights,test_accuracy)


Shape of trainingData: (750, 4) 
 Shape of trainingLabel (750, 1)
Shape of testData: (250, 4) 
 Shape of testLabel (250, 1)
All data: 1 uniform 0.792
All data: 1 distance 0.792
All data: 2 uniform 0.736
All data: 2 distance 0.792
All data: 3 uniform 0.776
All data: 3 distance 0.76
All data: 4 uniform 0.712
All data: 4 distance 0.768
All data: 5 uniform 0.736
All data: 5 distance 0.736
All data: 6 uniform 0.72
All data: 6 distance 0.764
All data: 7 uniform 0.724
All data: 7 distance 0.744
All data: 8 uniform 0.708
All data: 8 distance 0.728
All data: 9 uniform 0.74
All data: 9 distance 0.736
1 uniform 0.804
1 distance 0.804
2 uniform 0.832
2 distance 0.804
3 uniform 0.852
3 distance 0.824
4 uniform 0.86
4 distance 0.844
5 uniform 0.856
5 distance 0.836
6 uniform 0.864
6 distance 0.852
7 uniform 0.872
7 distance 0.86
8 uniform 0.852
8 distance 0.86
9 uniform 0.844
9 distance 0.856


* 使用trainingData所有数据，准确率最高为0.792
* 仅使用trainingData的加速度数据，准确率最高的为：n_neighbors=7,weights='uniform',准确率为：0.872